## 競艇データ 2022/11/01~2022/12/31


### データファイル変換

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
## データを読み込む
import os
import glob
FOLDER_PATH = '/content/drive/MyDrive/データサイエンス仙台ラボ/310. 各研修会・勉強会/001.一般向けConnpass勉強会/002. データ分析/第3回/DATA/'
file_paths = glob.glob(FOLDER_PATH+'*.lzh')

In [18]:
target_files_path = [i for i in file_paths if 'lzh' in i ] # lzh拡張子のファイルを含むファイルを対象にリスト化

In [20]:
len(target_files_path) # 格納されたファイル数を確認

122

In [21]:
# 番組表と結果表をそれぞれ分類する
bangumi_file_paths = []
kekka_file_paths = []
for target_file in target_files_path:
  if target_file[-11:-10]=='b':
    bangumi_file_paths.append(target_file)
  elif target_file[-11:-10]=='k':
    kekka_file_paths.append(target_file)
  else:
    pass

In [22]:
len(bangumi_file_paths), len(kekka_file_paths)

(61, 61)

In [23]:
list_b = [ i[-10:-4] for i in bangumi_file_paths]
list_k = [ i[-10:-4] for i in kekka_file_paths]

In [24]:
!pip install lhafile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 KB 2.6 MB/s eta 0:00:00


In [25]:
import lhafile

# 解凍したファイルを保存する場所を指定
TXT_BANNGUMI_DIR = "/content/drive/MyDrive/データサイエンス仙台ラボ/310. 各研修会・勉強会/001.一般向けConnpass勉強会/002. データ分析/第3回/DATA/banngumi_txt/"
TXT_KEKKA_DIR = "/content/drive/MyDrive/データサイエンス仙台ラボ/310. 各研修会・勉強会/001.一般向けConnpass勉強会/002. データ分析/第3回/DATA/kekka_txt/"

# ファイルを格納するフォルダを作成
os.makedirs(TXT_BANNGUMI_DIR, exist_ok=True)
os.makedirs(TXT_KEKKA_DIR, exist_ok=True)

# LZHファイルを解凍して任意のフォルダに格納する
# BANNGUMI
for target_file_path in bangumi_file_paths:

  target_file = lhafile.Lhafile(target_file_path)
  
  # 解凍したファイルの名前を取得
  info_file = target_file.infolist()
  file_name = info_file[0].filename

  # 解凍したファイルの保存
  open(TXT_BANNGUMI_DIR + file_name, "wb").write(target_file.read(file_name))

# LZHファイルを解凍して任意のフォルダに格納する
# KEKKA
for target_file_path in kekka_file_paths:

  target_file = lhafile.Lhafile(target_file_path)
  
  # 解凍したファイルの名前を取得
  info_file = target_file.infolist()
  file_name = info_file[0].filename

  # 解凍したファイルの保存
  open(TXT_KEKKA_DIR + file_name, "wb").write(target_file.read(file_name))

### データ抽出

In [31]:
# それぞれのデータフォルダ指定
TXT_BANNGUMI_DIR = "/content/drive/MyDrive/データサイエンス仙台ラボ/310. 各研修会・勉強会/001.一般向けConnpass勉強会/002. データ分析/第3回/DATA/banngumi_txt/"
TXT_KEKKA_DIR = "/content/drive/MyDrive/データサイエンス仙台ラボ/310. 各研修会・勉強会/001.一般向けConnpass勉強会/002. データ分析/第3回/DATA/kekka_txt/"

In [32]:
# TXTに変換したファイルを番組表、結果表とともに取得する
import glob
target_banngumi_file_paths_sorted = sorted(glob.glob(TXT_BANNGUMI_DIR+'*.TXT'))
target_kekka_file_paths_sorted = sorted(glob.glob(TXT_KEKKA_DIR+'*.TXT'))

In [33]:
target_banngumi_file_paths_sorted = [i for i in target_banngumi_file_paths_sorted if 'TXT' in i ]
target_kekka_file_paths_sorted = [i for i in target_kekka_file_paths_sorted if 'TXT' in i ]

In [34]:
len(target_banngumi_file_paths_sorted)

61

In [35]:
target_kekka_file_paths_sorted[0]

'/content/drive/MyDrive/データサイエンス仙台ラボ/310. 各研修会・勉強会/001.一般向けConnpass勉強会/002. データ分析/第3回/DATA/kekka_txt/K221101.TXT'

In [36]:
# 確か２４カ所ある、ボートレース場
place_code = {'桐生':'01',
              '戸田':'02',
              '江戸川':'03',
              '平和島':'04',
              '多摩川':'05',
              '浜名湖':'06',
              '蒲郡':'07',
              '常滑':'08',
              '津':'09',
              '三国':'10',
              'びわこ':'11',
              '住之江':'12',
              '尼崎':'13',
              '鳴門':'14',
              '丸亀':'15',
              '児島':'16',
              '宮島':'17',
              '徳山':'18',
              '下関':'19',
              '若松':'20',
              '芦屋':'21',
              '福岡':'22',
              '唐津':'23',
              '大村':'24'
              }

In [37]:
# 再起的に処理するため、関数化しておく　→ ファイル開く用
def open_text_file(file_path):
  with open(file_path, encoding='shift-jis') as f:
    data = f.readlines()
  return data

In [38]:
# テキストファイル名から年月日に変換する関数
def extract_date_from_text_file(file_path):
  return "20"+str(file_path[-10:-4])

In [39]:
import re
import pandas as pd

# 出走表から必要なデータ要素を抽出する関数
def extract_syussou_to_dataframe(data):
  #出走表を取り出す
  racers = [row.replace('\u3000','').replace('\n','') for row in data if re.match('^[1-6]\s', row)]

  #必要な13種類のデータを取り出す
  pattern = '^([1-6])\s(\d{4})([^0-9]+)(\d{2})([^0-9]+)(\d{2})([AB]\d{1})\s(\d{1}.\d{2})\s*(\d+.\d{2})\s(\d{1}.\d{2})\s*(\d+.\d{2})\s+\d+\s+(\d+.\d{2})\s*\d+\s+(\d+.\d{2})'
  pattern_re = re.compile(pattern)
  values = [re.match(pattern_re, racer).groups() for racer in racers]

  #取り出したデータをデータフレームに入れる
  column = ['艇番', '選手登番', '選手名', '年齢', '支部', '体重', '級別', '全国勝率', '全国2連率', '当地勝率', '当地2連率', 'モーター2連率', 'ボート2連率']
  df = pd.DataFrame(values,columns=column)

  return df

In [40]:
# ボートレースを含む文字列から、改行コードを消去する関数
def extract_race_place(data):
  return [row.replace('\u3000','').replace('\n','') for row in data if re.match('ボートレース', row)]

In [41]:
# 全ての文字列から、改行コードを消去する関数
def extract_all_text(data):
  return [row.replace('\u3000','').replace('\n','') for row in data]

In [42]:
# 会場ごとテキストを抽出するための、行数カウンター
def check_race_table_with_race_place(all_text, places):
  counter_list = []
  counter=0
  for text in all_text:
    for place in places:
      if place in text:
        counter_list.append(counter)
      else:
        pass
    counter +=1
  return counter_list

In [43]:
# レース番号を抽出する関数
def extract_race_place_name(places):
  return [ i[6:9].replace(" ", "") for i in places ]

In [44]:
# 出走表から必要なデータを抽出して、データフレームとして返す関数
def extract_syussou_to_dataframe(data):
  column = ['艇番',
            '選手登番',
            '選手名',
            '年齢',
            '支部',
            '体重',
            '級別',
            '全国勝率',
            '全国2連率',
            '当地勝率',
            '当地2連率',
            'モーター2連率',
            'ボート2連率',
            '会場'
            ]
  main_df = pd.DataFrame(columns=column)
  all_text = extract_all_text(data)
  places = extract_race_place(data)
  places_names = extract_race_place_name(places)
  counter_list = check_race_table_with_race_place(all_text, places)

  for num, places_name in zip(range(len(counter_list)), places_names):

    if num+2 <= len(counter_list):

      #出走表を取り出す
      target_data = all_text[counter_list[num]:counter_list[num+1]]

      racers = [row.replace('\u3000','').replace('\n','') for row in target_data if re.match('^[1-6]\s', row)]

      #必要な13種類のデータを取り出す
      pattern = '^([1-6])\s(\d{4})([^0-9]+)(\d{2})([^0-9]+)(\d{2})([AB]\d{1})\s*(\d+.\d{2})\s*(\d+.\d{2})\s*(\d+.\d{2})\s*(\d+.\d{2})\s+\d+\s*(\d+.\d{2})\s*\d+\s*(\d+.\d{2})'
      pattern_re = re.compile(pattern)
      values = [re.match(pattern_re, racer).groups() for racer in racers]

      #取り出したデータをデータフレームに入れる
      column = ['艇番', '選手登番', '選手名', '年齢', '支部', '体重', '級別', '全国勝率', '全国2連率', '当地勝率', '当地2連率', 'モーター2連率', 'ボート2連率']
      df = pd.DataFrame(values,columns=column)
      df['会場'] = places_name

      main_df = pd.concat([main_df, df])
    
    else:
    
      #出走表を取り出す

      target_data = all_text[counter_list[num]:]

      racers = [row.replace('\u3000','').replace('\n','') for row in target_data if re.match('^[1-6]\s', row)]

      #必要な13種類のデータを取り出す
      pattern = '^([1-6])\s(\d{4})([^0-9]+)(\d{2})([^0-9]+)(\d{2})([AB]\d{1})\s*(\d+.\d{2})\s*(\d+.\d{2})\s*(\d+.\d{2})\s*(\d+.\d{2})\s+\d+\s*(\d+.\d{2})\s*\d+\s*(\d+.\d{2})'
      pattern_re = re.compile(pattern)
      values = [re.match(pattern_re, racer).groups() for racer in racers]

      #取り出したデータをデータフレームに入れる
      column = ['艇番', '選手登番', '選手名', '年齢', '支部', '体重', '級別', '全国勝率', '全国2連率', '当地勝率', '当地2連率', 'モーター2連率', 'ボート2連率']
      df = pd.DataFrame(values,columns=column)
      df['会場'] = places_names[-1]

      main_df = pd.concat([main_df, df])

      break
  return main_df.reset_index(drop=True)

In [45]:
# データフレームにある"艇番", "会場"のみを返す関数
def extract_boot_num_race_place_from_main_df(main_df):
  return main_df[["艇番", "会場"]]

In [46]:
# レース会場ごとのレース番号にフラグをつけて返す関数
def replace_race_place_to_race_id(race_id_based_data):
  race_border_number=1
  place_code_flag=[]
  counter=0
  for num_boot, race_place in zip(race_id_based_data["艇番"], race_id_based_data["会場"]):
    if counter > 16:
      counter=0
      race_border_number+=1
      place_code_flag.append(int(place_code[race_place.replace(" ","")]))
    else:
      counter+=int(num_boot)
      place_code_flag.append(int(place_code[race_place.replace(" ","")]))
  return place_code_flag

In [50]:
# レース
def create_race_place_border_flag_from_race_id_based_data(race_id_based_data):
  race_place_border_flag = []
  counter = 1
  try:
    for i in range(len(race_id_based_data)):
      if int(race_id_based_data["race_place"][i:i+1]) != int(race_id_based_data["race_place"][i+1:i+2]):
        race_place_border_flag.append(counter)
        counter+=1
      else:
        race_place_border_flag.append(counter)
  except:
    race_place_border_flag.append(counter)
  
  return race_place_border_flag

In [47]:
# レースの番号(1~6)を区切る関数
def create_race_id_from_race_id_based_data(race_id_based_data):
  flag_list=[]
  race_border_number_flag=[]
  counter=0
  race_border_number=1
  for record_index in race_id_based_data.values:
    flag_list.append(record_index[3])
    if len(set(flag_list)) != int(1):
      flag_list=[]
      race_border_number=0
      race_border_number_flag.append(race_border_number+1)
    else:
      if counter > 16:
        counter=0
        race_border_number+=1
        race_border_number_flag.append(race_border_number)
      else:
        counter+=int(record_index[0])
        race_border_number_flag.append(race_border_number)
  return race_border_number_flag

In [48]:
# レースID(yyyymmdd + Pxx + Ryy)を作成する関数
def create_race_id_name(race_id_based_data, file_path):
  race_id_name=[]
  date_race=extract_date_from_text_file(file_path)
  for record_index in race_id_based_data.values:
    if len(str(record_index[4])) != int(2):
      if len(str(record_index[2])) != int(2):
        race_id_name.append(date_race+"P0"+str(record_index[2])+"R0"+str(record_index[4]))
      else:
        race_id_name.append(date_race+"P"+str(record_index[2])+"R0"+str(record_index[4]))
    else:
      if len(str(record_index[2])) != int(2):
        race_id_name.append(date_race+"P0"+str(record_index[2])+"R"+str(record_index[4]))
      else:
        race_id_name.append(date_race+"P"+str(record_index[2])+"R"+str(record_index[4]))
  return race_id_name

In [52]:
for target_file_path in target_banngumi_file_paths_sorted:
  try:
    data = open_text_file(target_file_path)

    places = extract_race_place(data)
    all_text = extract_all_text(data)
    counter_list = check_race_table_with_race_place(all_text, places)
    places_names = extract_race_place_name(places)

    main_df = extract_syussou_to_dataframe(data)

    race_id_based_data = extract_boot_num_race_place_from_main_df(main_df)
    race_id_based_data["race_place"] = replace_race_place_to_race_id(race_id_based_data)
    race_id_based_data["race_place_border_flag"] = create_race_place_border_flag_from_race_id_based_data(race_id_based_data)
    race_id_based_data["race_id"] = create_race_id_from_race_id_based_data(race_id_based_data)

    main_df["race_id"] = create_race_id_name(race_id_based_data, target_file_path)
    #print(main_df["race_id"][0])

    main_df.to_pickle(f'/content/drive/MyDrive/データサイエンス仙台ラボ/310. 各研修会・勉強会/001.一般向けConnpass勉強会/002. データ分析/第3回/DATA/banngumi_pickle/{target_file_path[-11:-4]}.pkl')

  except:
    print(target_file_path[-11:-4])

<ipython-input-52-cbcd8b941353>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  race_id_based_data["race_place"] = replace_race_place_to_race_id(race_id_based_data)
<ipython-input-52-cbcd8b941353>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  race_id_based_data["race_place_border_flag"] = create_race_place_border_flag_from_race_id_based_data(race_id_based_data)
<ipython-input-52-cbcd8b941353>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

**結果表からのデータ抽出**

In [53]:
def extract_kekkas(data):
  return [row.replace('\u3000','').replace('\n','') for row in data]

In [54]:
def extract_base_result(kekkas):
  # 抽出したい文字列を、パターンごとに整理する
  pattern_01= '^\s*(\d{2})\s*([0-6])\s*(\d{4})\s*([^0-9]+)\s*(\d{2})\s*(\d{2})\s*(\d{1}.\d{2})\s*([0-6])\s*(\d{1}.\d{2})\s*([ 0-9]+.[ 0-9]+.[ 0-9]+)' #レース・選手結果情報
  pattern_re_01 = re.compile(pattern_01)

  values_01=[]

  for kekka in kekkas:
    if re.match(pattern_re_01, kekka) == None:
      pass
    else:
      values_01.append(re.match(pattern_re_01, kekka).groups())

  return values_01

In [56]:
for target_file_path in target_kekka_file_paths_sorted:
  try:
    data = open_text_file(target_file_path)

    kekkas = extract_kekkas(data)

    pattern_03= '^\s*(\d{2}[A-Z]{4})'
    pattern_re_03 = re.compile(pattern_03)
    values_03=[]

    for kekka in kekkas:
      if re.match(pattern_re_03, kekka) == None:
        pass
      else:
        values_03.append(re.match(pattern_re_03, kekka).groups())

    place_flag=[]
    for i in values_03:
      if "BGN" in i[0]:
        place_flag.append(i)
    
    counter=0
    flag=[]
    for i in kekkas:
      for j in place_flag:
        if j[0] == i:
          flag.append(counter)
      counter+=1

    place_code=[]
    for i in place_flag:
      place_code.append(i[0][0:2])

    values_01=[]
    values_04=[]
    pattern_01= '^\s*(\d{2})\s*([0-6])\s*(\d{4})\s*([^0-9]+)\s*(\d{2})\s*(\d{2})\s*(\d{1}.\d{2})\s*([0-6])\s*(\d{1}.\d{2})\s*([ 0-9]+.[ 0-9]+.[ 0-9]+)' #レース・選手結果情報
    pattern_re_01 = re.compile(pattern_01)
    for i, j in zip(range(len(flag)), range(len(place_code))):
      try:
        for kekka in kekkas[flag[i]:flag[i+1]]:
          if re.match(pattern_re_01, kekka) == None:
            pass
          else:
            values_01.append(re.match(pattern_re_01, kekka).groups())
            values_04.append(place_code[j])
      except:
        for kekka in kekkas[flag[i]:]:
          if re.match(pattern_re_01, kekka) == None:
            pass
          else:
            values_01.append(re.match(pattern_re_01, kekka).groups())
            values_04.append(place_code[j])
        pass

    values_01_boot_num = [int(value[0]) for value in values_01]
    values_01_order_of_arrival = [int(value[1]) for value in values_01]
    values_01_player_id = [value[2] for value in values_01]
    values_01_player_name = [value[3] for value in values_01]
    values_01_time = [value[6] for value in values_01]

    column = ['艇番', '着順', '選手登番', '選手名', '展示']
    df = pd.DataFrame(columns=column)

    df['艇番'] = values_01_boot_num
    df['着順'] = values_01_order_of_arrival
    df['選手登番'] = values_01_player_id
    df['選手名'] = values_01_player_name
    df['展示'] = values_01_time
    df['place_code'] = values_04

    pattern_re_02_2='^\s*(\d{1,2}[A-Z])\s*(\D{1,30})\s*([A-Z]{1}\d{4}[a-z]{1})\s*(\D{1,2})\s*(\D{1,2})\s*(\D{1,4})\s*(\d{1,2}[a-z]{1})\s*(\D{1,4})\s*(\d{1,3}[a-z]{2})'
    values_02_2=[]
    counter=0
    flag=[]
    for kekka in kekkas:
      if re.match(pattern_re_02_2, kekka) == None:
        pass
      else:
        values_02_2.append(re.match(pattern_re_02_2, kekka).groups())
        flag.append(counter)
      counter+=1

    values_01_1=[]
    values_04_1=[]
    for i, j in zip(range(len(flag)), range(len(values_02_2))):
      try:
        for kekka in kekkas[flag[i]:flag[i+1]]:
          if re.match(pattern_re_01, kekka) == None:
            pass
          else:
            values_01_1.append(re.match(pattern_re_01, kekka).groups())
            values_04_1.append(values_02_2[j])
      except:
        for kekka in kekkas[flag[i]:]:
          if re.match(pattern_re_01, kekka) == None:
            pass
          else:
            values_01_1.append(re.match(pattern_re_01, kekka).groups())
            values_04_1.append(values_02_2[j])
        pass
    
    race_id_num=[]
    race_weather=[]
    race_wind_arrow=[]
    race_wind_strongth=[]
    race_wave=[] 

    for i in values_04_1:
      race_id_num.append(i[0][:-1])
      race_weather.append(i[3].replace(" ",""))
      race_wind_arrow.append(i[5].replace(" ",""))
      race_wind_strongth.append(i[6][:-1].replace(" ",""))
      race_wave.append(i[8][:-2].replace(" ",""))

    df["race_num"]=race_id_num
    df["天候"]=race_weather
    df["風向"]=race_wind_arrow
    df["風速"]=race_wind_strongth
    df["波高"]=race_wave

    race_id_list=[]
    for i, j in zip([int(k) for k in df["place_code"].to_list()], [int(l) for l in df["race_num"].to_list()]):

        if len(str(i)) != int(2):
          if len(str(j)) != int(2):
            race_id_list.append("20"+target_file_path[-10:-4]+"P0"+str(i)+"R0"+str(j))
          else:
            race_id_list.append("20"+target_file_path[-10:-4]+"P0"+str(i)+"R"+str(j))
        else:
          if len(str(j)) != int(2):
            race_id_list.append("20"+target_file_path[-10:-4]+"P"+str(i)+"R0"+str(j))
          else:
            race_id_list.append("20"+target_file_path[-10:-4]+"P"+str(i)+"R"+str(j))

    df["race_id"]=race_id_list

    df.to_pickle(f'/content/drive/MyDrive/データサイエンス仙台ラボ/310. 各研修会・勉強会/001.一般向けConnpass勉強会/002. データ分析/第3回/DATA/kekka_pickle/{target_file_path[-11:-4]}.pkl')

  except:
    print(target_file_path[-11:-4])

K221112
K221113
K221114
K221115
K221221


### 検証データ整形
2023/01/02, 2023/01/03, 2023/01/08は除外

In [57]:
# それぞれのデータフォルダ指定
TXT_BANNGUMI_DIR = "/content/drive/MyDrive/データサイエンス仙台ラボ/310. 各研修会・勉強会/001.一般向けConnpass勉強会/002. データ分析/第3回/DATA/banngumi_pickle/"
TXT_KEKKA_DIR = "/content/drive/MyDrive/データサイエンス仙台ラボ/310. 各研修会・勉強会/001.一般向けConnpass勉強会/002. データ分析/第3回/DATA/kekka_pickle/"

In [58]:
import glob
target_banngumi_file_paths_sorted = sorted(glob.glob(TXT_BANNGUMI_DIR+'*.pkl'))
target_kekka_file_paths_sorted = sorted(glob.glob(TXT_KEKKA_DIR+'*.pkl'))

In [59]:
target_banngumi_file_paths_sorted[0]

'/content/drive/MyDrive/データサイエンス仙台ラボ/310. 各研修会・勉強会/001.一般向けConnpass勉強会/002. データ分析/第3回/DATA/banngumi_pickle/B221101.pkl'

In [60]:
target_kekka_file_paths_sorted[0]

'/content/drive/MyDrive/データサイエンス仙台ラボ/310. 各研修会・勉強会/001.一般向けConnpass勉強会/002. データ分析/第3回/DATA/kekka_pickle/K221101.pkl'

In [61]:
# 結果表から、一部データを取得することができなかった
len(target_banngumi_file_paths_sorted), len(target_kekka_file_paths_sorted)

(61, 56)

In [2]:
import numpy as np
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

In [63]:
df_from_pkl_init = pd.read_pickle(target_banngumi_file_paths_sorted[0])

In [64]:
for target_file in target_banngumi_file_paths_sorted[1:]:
  
  df_from_pkl_tmp = pd.read_pickle(target_file)
  df_from_pkl_init = pd.concat([df_from_pkl_init, df_from_pkl_tmp])

In [65]:
df_from_pkl_init.columns

Index(['艇番', '選手登番', '選手名', '年齢', '支部', '体重', '級別', '全国勝率', '全国2連率', '当地勝率',
       '当地2連率', 'モーター2連率', 'ボート2連率', '会場', 'race_id'],
      dtype='object')

In [66]:
df_from_pkl_init.sort_values('race_id', inplace=True)

In [67]:
df_from_pkl_init = df_from_pkl_init.reset_index(drop=True)

In [68]:
df_from_pkl_init.to_pickle("/content/drive/MyDrive/データサイエンス仙台ラボ/310. 各研修会・勉強会/001.一般向けConnpass勉強会/002. データ分析/第3回/DATA/kennsyouyou_banngumi.pkl")

In [69]:
df_from_pkl_init = pd.read_pickle(target_kekka_file_paths_sorted[0])

In [70]:
for target_file in target_kekka_file_paths_sorted[1:]:
  
  df_from_pkl_tmp = pd.read_pickle(target_file)
  df_from_pkl_init = pd.concat([df_from_pkl_init, df_from_pkl_tmp])

In [71]:
df_from_pkl_init.sort_values('race_id', inplace=True)

In [72]:
df_from_pkl_init = df_from_pkl_init.reset_index(drop=True)

In [73]:
df_from_pkl_init.to_pickle("/content/drive/MyDrive/データサイエンス仙台ラボ/310. 各研修会・勉強会/001.一般向けConnpass勉強会/002. データ分析/第3回/DATA/kennsyouyou_kekka.pkl")

**番組表と結果表を結合する**

In [74]:
df_kekka = pd.read_pickle("/content/drive/MyDrive/データサイエンス仙台ラボ/310. 各研修会・勉強会/001.一般向けConnpass勉強会/002. データ分析/第3回/DATA/kennsyouyou_kekka.pkl")
df_banngumi = pd.read_pickle("/content/drive/MyDrive/データサイエンス仙台ラボ/310. 各研修会・勉強会/001.一般向けConnpass勉強会/002. データ分析/第3回/DATA/kennsyouyou_banngumi.pkl")

In [75]:
df_all = pd.concat([df_banngumi, df_kekka])

In [76]:
df_all = pd.merge(df_banngumi, df_kekka, on=['選手登番', 'race_id'], how='left')

In [77]:
df_all

,艇番_x,選手登番,選手名_x,年齢,支部,体重,級別,全国勝率,全国2連率,当地勝率,...,艇番_y,着順,選手名_y,展示,place_code,race_num,天候,風向,風速,波高
0,5,4058,野末智一,43,静岡,52,B1,5.08,30.77,4.53,...,6.0,5.0,野末智一,6.85,01,1,曇り,西,1,1
1,6,4399,松本晶恵,35,群馬,46,A2,6.68,51.67,7.51,...,2.0,6.0,松本晶恵,6.80,01,1,曇り,西,1,1
2,4,5051,松本純平,27,埼玉,55,B1,6.12,42.27,4.97,...,1.0,4.0,松本純平,6.66,01,1,曇り,西,1,1
3,1,4389,伊藤喜智,37,大阪,52,B1,4.34,22.11,4.21,...,4.0,1.0,伊藤喜智,6.74,01,1,曇り,西,1,1
4,2,3951,石川吉鎬,46,大阪,56,A2,5.67,34.71,6.20,...,3.0,2.0,石川吉鎬,6.72,01,1,曇り,西,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55412,2,3815,森岡満郎,46,岡山,55,B1,4.99,30.63,5.64,...,3.0,2.0,森岡満郎,6.85,24,12,晴,無風,0,1
55413,3,5022,伊藤尚汰,28,福岡,48,B1,3.58,14.08,3.58,...,2.0,3.0,伊藤尚汰,6.79,24,12,晴,無風,0,1
55414,4,3928,林恵祐,47,福岡,58,B1,4.27,25.42,4.11,...,6.0,4.0,林恵祐,6.90,24,12,晴,無風,0,1
55415,5,3564,桑原啓,52,福岡,54,B1,5.09,32.09,4.79,...,5.0,5.0,桑原啓,6.78,24,12,晴,無風,0,1


In [78]:
df_all = df_all.drop(['艇番_x', '選手名_x', '支部', '選手名_y', 'place_code', 'race_num'],axis=1) # 重複するカラムや不要なカラムを除く
df_all = df_all.dropna(how='any') # 欠損を含むレコードは削除する（悪天候でレースが開催できなかったなど）
df_all = df_all.reset_index(drop=True)

In [80]:
df_all[df_all['艇番_y']==0.0]

,選手登番,年齢,体重,級別,全国勝率,全国2連率,当地勝率,当地2連率,モーター2連率,ボート2連率,会場,race_id,艇番_y,着順,展示,天候,風向,風速,波高
4104,3551,51,46,B1,5.03,23.73,4.97,23.68,34.53,32.06,三国,20221106P10R08,0.0,6.0,6.60,晴,無風,0,0
28467,3731,49,53,B1,4.81,30.34,4.98,32.08,41.79,32.50,児島,20221213P16R10,0.0,2.0,6.99,晴,北西,2,3


In [81]:
df_all[4100:4110]

,選手登番,年齢,体重,級別,全国勝率,全国2連率,当地勝率,当地2連率,モーター2連率,ボート2連率,会場,race_id,艇番_y,着順,展示,天候,風向,風速,波高
4100,4819,31,47,B1,4.93,24.69,4.62,24.14,31.82,38.19,三国,20221106P10R07,4.0,4.0,6.75,晴,東,1,1
4101,4844,29,48,B1,3.75,15.09,4.50,16.67,35.42,35.46,三国,20221106P10R07,3.0,3.0,6.74,晴,東,1,1
4102,5168,22,48,B2,2.28,3.28,1.29,0.00,34.06,28.78,三国,20221106P10R07,6.0,2.0,6.68,晴,東,1,1
4103,4548,34,50,B1,4.19,14.58,0.00,0.00,40.15,34.97,三国,20221106P10R07,1.0,1.0,6.65,晴,東,1,1
4104,3551,51,46,B1,5.03,23.73,4.97,23.68,34.53,32.06,三国,20221106P10R08,0.0,6.0,6.60,晴,無風,0,0
4105,5144,21,49,B1,2.17,4.88,1.60,0.00,39.07,36.23,三国,20221106P10R09,6.0,6.0,6.63,晴,北西,2,2
4106,5088,22,46,A2,5.65,39.58,5.78,44.44,32.88,30.07,三国,20221106P10R09,1.0,4.0,6.74,晴,北西,2,2
4107,4627,36,48,A2,6.93,58.62,7.45,64.52,26.06,34.06,三国,20221106P10R09,2.0,5.0,6.63,晴,北西,2,2
4108,4372,35,47,A1,6.24,45.00,6.79,48.48,31.51,34.81,三国,20221106P10R09,4.0,2.0,6.73,晴,北西,2,2
4109,4510,33,49,B2,3.70,13.04,0.00,0.00,31.08,43.70,三国,20221106P10R09,3.0,3.0,6.76,晴,北西,2,2


In [82]:
df_all[28463:28473]

,選手登番,年齢,体重,級別,全国勝率,全国2連率,当地勝率,当地2連率,モーター2連率,ボート2連率,会場,race_id,艇番_y,着順,展示,天候,風向,風速,波高
28463,5075,27,53,A2,6.13,39.22,2.11,0.00,29.65,41.31,児島,20221213P16R08,4.0,2.0,6.87,晴,北西,2,3
28464,4697,36,49,B1,4.31,23.89,4.33,25.93,33.92,28.47,児島,20221213P16R08,2.0,3.0,6.79,晴,北西,2,3
28465,3964,43,57,B1,4.06,21.51,4.26,18.46,29.41,34.98,児島,20221213P16R08,6.0,1.0,6.89,晴,北西,2,3
28466,4824,29,52,A2,6.28,44.66,0.00,0.00,37.34,34.27,児島,20221213P16R08,3.0,4.0,6.89,晴,北西,2,3
28467,3731,49,53,B1,4.81,30.34,4.98,32.08,41.79,32.50,児島,20221213P16R10,0.0,2.0,6.99,晴,北西,2,3
28468,3333,54,53,A1,6.19,45.83,5.64,35.85,32.34,33.33,児島,20221213P16R11,1.0,1.0,6.83,晴,北西,2,3
28469,4067,42,53,A1,6.08,37.12,6.24,33.33,33.91,35.20,児島,20221213P16R11,5.0,2.0,6.79,晴,北西,2,3
28470,4759,33,52,A2,6.07,37.50,6.36,45.45,42.06,33.17,児島,20221213P16R11,4.0,3.0,6.88,晴,北西,2,3
28471,4471,35,52,A2,6.22,43.64,6.83,61.90,29.36,32.89,児島,20221213P16R11,2.0,4.0,6.83,晴,北西,2,3
28472,5016,24,53,A2,6.34,43.88,3.22,11.11,28.33,25.00,児島,20221213P16R11,3.0,5.0,6.99,晴,北西,2,3


In [79]:
# 艇番_y がゼロのものは除く
df_all_1 = df_all[df_all['艇番_y']!=0.0]

In [83]:
df_all_1

,選手登番,年齢,体重,級別,全国勝率,全国2連率,当地勝率,当地2連率,モーター2連率,ボート2連率,会場,race_id,艇番_y,着順,展示,天候,風向,風速,波高
0,4058,43,52,B1,5.08,30.77,4.53,26.67,30.73,32.51,桐生,20221101P01R01,6.0,5.0,6.85,曇り,西,1,1
1,4399,35,46,A2,6.68,51.67,7.51,67.39,45.23,34.60,桐生,20221101P01R01,2.0,6.0,6.80,曇り,西,1,1
2,5051,27,55,B1,6.12,42.27,4.97,30.56,58.08,42.45,桐生,20221101P01R01,1.0,4.0,6.66,曇り,西,1,1
3,4389,37,52,B1,4.34,22.11,4.21,21.05,36.47,35.71,桐生,20221101P01R01,4.0,1.0,6.74,曇り,西,1,1
4,3951,46,56,A2,5.67,34.71,6.20,50.00,30.29,32.08,桐生,20221101P01R01,3.0,2.0,6.72,曇り,西,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42942,3815,46,55,B1,4.99,30.63,5.64,30.30,20.99,48.78,大村,20221231P24R12,3.0,2.0,6.85,晴,無風,0,1
42943,5022,28,48,B1,3.58,14.08,3.58,16.67,32.26,20.78,大村,20221231P24R12,2.0,3.0,6.79,晴,無風,0,1
42944,3928,47,58,B1,4.27,25.42,4.11,24.44,34.74,28.89,大村,20221231P24R12,6.0,4.0,6.90,晴,無風,0,1
42945,3564,52,54,B1,5.09,32.09,4.79,25.00,28.87,27.38,大村,20221231P24R12,5.0,5.0,6.78,晴,無風,0,1


**簡単に統計量を見てみる**


In [84]:
df_all_1.describe()

,艇番_y,着順
count,42945.000000,42945.000000
mean,3.467039,3.497380
std,1.696143,1.708838
min,1.000000,1.000000
25%,2.000000,2.000000
50%,3.000000,3.000000
75%,5.000000,5.000000
max,6.000000,6.000000


In [ ]:
df_all_1["選手登番"]=df_all_1["選手登番"].astype(int)
df_all_1["年齢"]=df_all_1["年齢"].astype(int)
df_all_1["体重"]=df_all_1["体重"].astype(int)
df_all_1["全国勝率"]=df_all_1["全国勝率"].astype(np.float16)
df_all_1["全国2連率"]=df_all_1["全国2連率"].astype(np.float16)
df_all_1["当地勝率"]=df_all_1["当地勝率"].astype(np.float16)
df_all_1["当地2連率"]=df_all_1["当地2連率"].astype(np.float16)
df_all_1["モーター2連率"]=df_all_1["モーター2連率"].astype(np.float16)
df_all_1["艇番_y"]=df_all_1["艇番_y"].astype(int)
df_all_1["着順"]=df_all_1["着順"].astype(int)
df_all_1["展示"]=df_all_1["展示"].astype(np.float16)
df_all_1["風速"]=df_all_1["風速"].astype(int)
df_all_1["波高"]=df_all_1["波高"].astype(int)

In [86]:
df_all_1_dummy = pd.get_dummies(df_all_1, columns=["級別", "会場", "艇番_y", "天候", "風向"])

In [87]:
df_all_1_dummy.columns

Index(['選手登番', '年齢', '体重', '全国勝率', '全国2連率', '当地勝率', '当地2連率', 'モーター2連率',
       'ボート2連率', 'race_id', '着順', '展示', '風速', '波高', '級別_A1', '級別_A2', '級別_B1',
       '級別_B2', '会場_びわこ', '会場_三国', '会場_下関', '会場_丸亀', '会場_住之江', '会場_児島',
       '会場_唐津', '会場_多摩川', '会場_大村', '会場_宮島', '会場_尼崎', '会場_常滑', '会場_平和島',
       '会場_徳山', '会場_戸田', '会場_桐生', '会場_江戸川', '会場_津', '会場_浜名湖', '会場_芦屋', '会場_若松',
       '会場_蒲郡', '会場_鳴門', '艇番_y_1', '艇番_y_2', '艇番_y_3', '艇番_y_4', '艇番_y_5',
       '艇番_y_6', '天候_晴', '天候_曇り', '天候_雨', '天候_雪', '天候_霧', '風向_北', '風向_北東',
       '風向_北西', '風向_南', '風向_南東', '風向_南西', '風向_東', '風向_無風', '風向_西'],
      dtype='object')

In [89]:
df_all_1_dummy.describe()

,選手登番,年齢,体重,全国勝率,全国2連率,当地勝率,当地2連率,モーター2連率,着順,展示,...,天候_霧,風向_北,風向_北東,風向_北西,風向_南,風向_南東,風向_南西,風向_東,風向_無風,風向_西
count,42945.000000,42945.000000,42945.000000,4.294500e+04,4.294500e+04,4.294500e+04,4.294500e+04,4.294500e+04,42945.000000,4.294500e+04,...,42945.000000,42945.000000,42945.000000,42945.000000,42945.000000,42945.000000,42945.000000,42945.000000,42945.000000,42945.000000
mean,4311.253487,38.509046,52.536360,inf,inf,inf,inf,inf,3.497380,inf,...,0.000140,0.173291,0.104366,0.240587,0.057283,0.069717,0.059425,0.090348,0.093142,0.111841
std,568.760407,10.124583,2.979811,1.330078e+00,1.428906e+01,1.979492e+00,1.821875e+01,1.068750e+01,1.708838,1.052246e-01,...,0.011819,0.378503,0.305738,0.427445,0.232384,0.254673,0.236421,0.286683,0.290635,0.315174
min,2014.000000,17.000000,43.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,6.250000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3898.000000,31.000000,51.000000,4.390625e+00,2.273438e+01,4.058594e+00,1.818750e+01,2.826562e+01,2.000000,6.718750e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4330.000000,38.000000,53.000000,5.269531e+00,3.300000e+01,5.191406e+00,3.151562e+01,3.309375e+01,3.000000,6.789062e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4791.000000,46.000000,54.000000,6.179688e+00,4.387500e+01,6.140625e+00,4.346875e+01,3.821875e+01,5.000000,6.859375e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5283.000000,75.000000,67.000000,8.476562e+00,7.943750e+01,1.088281e+01,1.000000e+02,8.887500e+01,6.000000,7.550781e+00,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [92]:
df_all_1_dummy.to_pickle('/content/drive/MyDrive/データサイエンス仙台ラボ/310. 各研修会・勉強会/001.一般向けConnpass勉強会/002. データ分析/第3回/DATA/kensyouyou_df_all_1_dummy.pkl')

## 訓練データの読み込みとLightGBMによる訓練

In [4]:
df_all_1_dummy = pd.read_pickle('/content/drive/MyDrive/データサイエンス仙台ラボ/310. 各研修会・勉強会/001.一般向けConnpass勉強会/002. データ分析/第3回/DATA/kensyouyou_df_all_1_dummy.pkl')

In [5]:
# 説明変数,目的変数
X = df_all_1_dummy.drop(['race_id', '着順'],axis=1).values # 説明変数(着順以外の特徴量)
y = df_all_1_dummy[['着順']].values

In [6]:
y = [i[0]-1 for i in y ] # 便宜上、1着 → 0 , ... , 6着　→　5とする

**パラメータチューニングとLightGBM GPUを利用できるよう設定**

In [1]:
!git clone --recursive https://github.com/Microsoft/LightGBM
%cd LightGBM
!mkdir build
%cd build
!cmake ../../LightGBM -DUSE_GPU=1
!make -j4
%cd ../python-package
!python3 setup.py install --gpu

Cloning into 'LightGBM'...
remote: Enumerating objects: 28804, done.
remote: Counting objects: 100% (3885/3885), done.
remote: Compressing objects: 100% (465/465), done.
remote: Total 28804 (delta 3601), reused 3514 (delta 3407), pack-reused 24919
Receiving objects: 100% (28804/28804), 20.19 MiB | 29.92 MiB/s, done.
Resolving deltas: 100% (21333/21333), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'external_libs/compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'external_libs/eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/external_libs/compute'...
remote: Enumerating objects: 21733, done.        
remote: Counting objects: 100% (5/5), done.        
remot

In [7]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 KB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 11.8 MB/s eta 0:00:00


In [8]:
import optuna

In [9]:
import warnings # 実行に関係ない警告を無視
warnings.filterwarnings('ignore')
import lightgbm as lgb #LightGBM
from sklearn import datasets
from sklearn.model_selection import train_test_split # データセット分割用
from sklearn.metrics import accuracy_score # モデル評価用(正答率)
from sklearn.metrics import log_loss # モデル評価用(logloss)     
from sklearn.metrics import roc_auc_score # モデル評価用(auc)

In [10]:
from sklearn.model_selection import KFold # 交差検証用

In [11]:
def objective(trial):
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'num_class': 6, # クラス数 : 6
        'metric': {'multi_error'},
        #'device': 'gpu',
        'verbosity': -1,
        "seed":42,
        "learning_rate":trial.suggest_loguniform('lambda_l1', 0.005, 0.03),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    num_round = 1000
    FOLD_NUM = 5

    models = []
    kf = KFold(n_splits=FOLD_NUM, random_state=42, shuffle=True)
    accs = []
    feature_importance_df = pd.DataFrame()


    #pred_cv = np.zeros(len(.index))

    for i, (tdx, vdx) in enumerate(kf.split(X, y)):
        print(f'Fold : {i}')
        tdx = [int(i) for i in tdx]
        vdx = [int(i) for i in vdx]
        X_train, X_valid, y_train, y_valid = X[tdx], X[vdx], np.array(y)[tdx], np.array(y)[vdx]
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_valid = lgb.Dataset(X_valid, y_valid)
        model = lgb.train(params, lgb_train, num_boost_round=num_round,
                      #categorical_feature=cat_features,
                      #shuffle=True,
                      valid_names=["train", "valid"], valid_sets=[lgb_train, lgb_valid],
                      callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=True)])
        va_pred = model.predict(X_valid)
        va_pred = np.argmax(va_pred,axis=1)
        acc_ = accuracy_score(y_valid,va_pred)
        print('Acc :', acc_)
        #score_ = -mean_squared_error(y_valid.values, va_pred)  # 改良の余地あり
        print(acc_)
        accs.append(acc_)
        models.append(model)

    return np.mean(accs)

In [12]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10) # 本来はn_trials=10000とか設定して探索的チューニングする

[I 2023-01-21 03:46:55,011] A new study created in memory with name: no-name-7bdaa6b1-6725-4ef2-865c-3d5a8fbb4072


Fold : 0
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[26]	train's multi_error: 0.667976	valid's multi_error: 0.712539
Acc : 0.2874607055536151
0.2874607055536151
Fold : 1
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[6]	train's multi_error: 0.68189	valid's multi_error: 0.715217
Acc : 0.2847828617999767
0.2847828617999767
Fold : 2
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[24]	train's multi_error: 0.664775	valid's multi_error: 0.701828
Acc : 0.2981720805681686
0.2981720805681686
Fold : 3
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[14]	train's multi_error: 0.683636	valid's multi_error: 0.713471
Acc : 0.2865292816393061
0.2865292816393061
Fold : 4
Training until validation scores don't improve for 10 rounds


[I 2023-01-21 03:47:02,929] Trial 0 finished with value: 0.2890674118057981 and parameters: {'lambda_l1': 0.007652028320199109, 'lambda_l2': 0.18460615009701964, 'num_leaves': 29, 'feature_fraction': 0.763218485980278, 'bagging_fraction': 0.6211648214867722, 'bagging_freq': 6, 'min_child_samples': 72}. Best is trial 0 with value: 0.2890674118057981.


Early stopping, best iteration is:
[5]	train's multi_error: 0.687274	valid's multi_error: 0.711608
Acc : 0.2883921294679241
0.2883921294679241
Fold : 0
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[22]	train's multi_error: 0.650978	valid's multi_error: 0.711142
Acc : 0.2888578414250786
0.2888578414250786
Fold : 1
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[19]	train's multi_error: 0.656392	valid's multi_error: 0.712772
Acc : 0.28722784957503783
0.28722784957503783
Fold : 2
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[17]	train's multi_error: 0.658691	valid's multi_error: 0.702876
Acc : 0.29712422866457094
0.29712422866457094
Fold : 3
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[23]	train's multi_error: 0.668588	valid's multi_error: 0.714519
Acc : 0.28548142973570845
0.2854814297357

[I 2023-01-21 03:47:10,032] Trial 1 finished with value: 0.2900454069158226 and parameters: {'lambda_l1': 0.005156369996216607, 'lambda_l2': 0.00015738161627424515, 'num_leaves': 39, 'feature_fraction': 0.8900058609007759, 'bagging_fraction': 0.7040124532314294, 'bagging_freq': 5, 'min_child_samples': 8}. Best is trial 1 with value: 0.2900454069158226.


...

Early stopping, best iteration is:
[3]	train's multi_error: 0.685062	valid's multi_error: 0.714751
Acc : 0.2852485737571312
0.2852485737571312
Fold : 0
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[6]	train's multi_error: 0.603504	valid's multi_error: 0.721039
Acc : 0.2789614623355455
0.2789614623355455
Fold : 1
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[7]	train's multi_error: 0.590057	valid's multi_error: 0.725579
Acc : 0.2744207707532891
0.2744207707532891
Fold : 2
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[9]	train's multi_error: 0.588456	valid's multi_error: 0.721271
Acc : 0.2787286063569682
0.2787286063569682
Fold : 3
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[7]	train's multi_error: 0.623414	valid's multi_error: 0.726511
Acc : 0.2734893468389801
0.2734893468389801
Fold 

[I 2023-01-21 03:48:05,894] Trial 8 finished with value: 0.2781697520083828 and parameters: {'lambda_l1': 0.00916343139573197, 'lambda_l2': 0.004104663154119207, 'num_leaves': 173, 'feature_fraction': 0.7226074919785818, 'bagging_fraction': 0.8417406779636327, 'bagging_freq': 2, 'min_child_samples': 91}. Best is trial 1 with value: 0.2900454069158226.


Early stopping, best iteration is:
[4]	train's multi_error: 0.624753	valid's multi_error: 0.714751
Acc : 0.2852485737571312
0.2852485737571312
Fold : 0
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[22]	train's multi_error: 0.64146	valid's multi_error: 0.715566
Acc : 0.2844335778321108
0.2844335778321108
Fold : 1
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[27]	train's multi_error: 0.634009	valid's multi_error: 0.715217
Acc : 0.2847828617999767
0.2847828617999767
Fold : 2
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[16]	train's multi_error: 0.644429	valid's multi_error: 0.707999
Acc : 0.2920013971358715
0.2920013971358715
Fold : 3
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[18]	train's multi_error: 0.659681	valid's multi_error: 0.717546
Acc : 0.28245430201420424
0.28245430201420424


[I 2023-01-21 03:48:12,690] Trial 9 finished with value: 0.2865292816393061 and parameters: {'lambda_l1': 0.006192597867948953, 'lambda_l2': 0.0009849201138008006, 'num_leaves': 54, 'feature_fraction': 0.9223170964501776, 'bagging_fraction': 0.6059824373827564, 'bagging_freq': 1, 'min_child_samples': 77}. Best is trial 1 with value: 0.2900454069158226.


Early stopping, best iteration is:
[13]	train's multi_error: 0.652841	valid's multi_error: 0.711026
Acc : 0.2889742694143672
0.2889742694143672


In [13]:
study.best_params

{'lambda_l1': 0.005156369996216607,
 'lambda_l2': 0.00015738161627424515,
 'num_leaves': 39,
 'feature_fraction': 0.8900058609007759,
 'bagging_fraction': 0.7040124532314294,
 'bagging_freq': 5,
 'min_child_samples': 8}

In [14]:
study.best_value

0.2900454069158226

**最適と思われるパラメータの1組みが決まったので...**

In [15]:
# LightGBM parameters
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 6, # クラス数 : 6
    'metric': {'multi_error'},
    'device': 'gpu',
    'verbosity': -1,
    "seed":42,
}

In [16]:
params.update(study.best_params)

In [17]:
params

{'task': 'train',
 'boosting_type': 'gbdt',
 'objective': 'multiclass',
 'num_class': 6,
 'metric': {'multi_error'},
 'device': 'gpu',
 'verbosity': -1,
 'seed': 42,
 'lambda_l1': 0.005156369996216607,
 'lambda_l2': 0.00015738161627424515,
 'num_leaves': 39,
 'feature_fraction': 0.8900058609007759,
 'bagging_fraction': 0.7040124532314294,
 'bagging_freq': 5,
 'min_child_samples': 8}

In [18]:
# トレーニングデータ,テストデータの分割
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=2)

In [19]:
# 学習に使用するデータを設定
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [21]:
# モデルの学習
model = lgb.train(params,
                  train_set=lgb_train, # トレーニングデータの指定
                  valid_sets=lgb_eval, # 検証データの指定
                  callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=True)]
                  )

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's multi_error: 0.716032


In [22]:
# テストデータの予測 ((各クラスの予測確率 [クラス0の予測確率,クラス1の予測確率,クラス2の予測確率] を返す))
y_pred_prob = model.predict(X_test)
# テストデータの予測 (予測クラス(0 or 1 or 2)を返す)
y_pred = np.argmax(y_pred_prob, axis=1) # 一番大きい予測確率のクラスを予測クラスに

In [23]:
y_pred_prob

array([[0.15990308, 0.16968458, 0.16770078, 0.16865679, 0.16724888,
        0.16680589],
       [0.15668546, 0.16381141, 0.16536073, 0.168591  , 0.16990382,
        0.17564758],
       [0.1883203 , 0.16620636, 0.16993665, 0.16180552, 0.15732621,
        0.15640497],
       ...,
       [0.15651362, 0.16696783, 0.16517937, 0.16706558, 0.16881866,
        0.17545493],
       [0.1578281 , 0.16662101, 0.17041728, 0.16846869, 0.17027487,
        0.16639005],
       [0.15646207, 0.16691284, 0.16350746, 0.16701056, 0.1748008 ,
        0.17130628]])

In [24]:
# モデル評価
# acc : 正答率
acc = accuracy_score(y_test,y_pred)
print('Acc :', acc)

Acc : 0.2839678658749563


**特徴量がモデルにどのように効いているかを確認する**

In [29]:
cols = list(df_all_1_dummy.drop(['race_id', '着順'],axis=1).columns)

In [30]:
f_importance = np.array(model.feature_importance(importance_type='gain')) # 特徴量重要度の算出
f_importance = f_importance / np.sum(f_importance)  # 正規化(必要ない場合はコメントアウト)
df_importance = pd.DataFrame({'feature':cols, 'importance':f_importance})
df_importance = df_importance.sort_values('importance', ascending=False) # 降順ソート

In [31]:
df_importance

,feature,importance
39,艇番_y_1,0.510395
0,選手登番,0.128187
40,艇番_y_2,0.094358
4,全国2連率,0.031488
41,艇番_y_3,0.030864
44,艇番_y_6,0.021974
3,全国勝率,0.021399
7,モーター2連率,0.018583
9,展示,0.018169
43,艇番_y_5,0.015760


**トレーニングできたモデルを保存**

In [32]:
import pickle

In [35]:
file_name = '/content/drive/MyDrive/データサイエンス仙台ラボ/310. 各研修会・勉強会/001.一般向けConnpass勉強会/002. データ分析/第3回/DATA//MODEL/trained_model_v1.pkl'
pickle.dump(model, open(file_name, 'wb'))

## 学習済みモデルの読み込み

In [37]:
model = pickle.load(open(file_name, 'rb'))